# Libraries


# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project-experiment'

experiment = Experiment(ws, experiment_name)

In [4]:
dataset = Dataset.get_by_name(ws, "default-ibk-dataset")
df = dataset.to_pandas_dataframe()

In [5]:
df.head()

,key_value,target,edad,sexo,est_cvl,sit_lab,ctd_hijos,flg_sin_email,ctd_veh,tip_lvledu,...,total_max_of_saldo_sum_per_month,total_min_of_saldo_min_per_month,total_mean_of_saldo_min_per_month,total_max_of_saldo_max_per_month,total_mean_of_saldo_max_per_month,total_min_of_condicion_min_per_month,total_mean_of_condicion_min_per_month,total_max_of_condicion_max_per_month,total_mean_of_condicion_max_per_month,total_mean_of_condicion_mean_per_month
0,0,0,-0.983651,1.0,4.0,2.0,0.0,1.0,0.0,0.0,...,6.829334,0.927141,0.936001,1.072672,1.014399,0,0.0,0,0.000,0.000000
1,1,0,0.000953,1.0,4.0,1.0,0.0,1.0,1.0,0.0,...,3.729381,0.927119,0.927238,0.938372,0.938051,0,0.0,0,0.000,0.000000
2,2,0,0.531124,0.0,4.0,1.0,0.0,1.0,0.0,7.0,...,2.849616,0.927119,0.927119,0.995254,0.964115,0,0.0,0,0.000,0.000000
3,3,0,-1.135129,0.0,4.0,1.0,0.0,1.0,0.0,7.0,...,4.076464,0.927209,0.928015,1.114270,1.104669,0,0.0,0,0.000,0.000000
4,4,1,-1.438084,0.0,4.0,1.0,0.0,1.0,0.0,0.0,...,11.229775,0.927220,0.928755,0.957766,0.952834,0,0.0,80,19.125,1.730303


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [7]:
project_folder = './capstone-project-pipeline'

In [60]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5,
    "max_cores_per_iteration": -1
    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-experiment,AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [62]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [54]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-experiment,AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|56007                            |1                                |354663                                |
+---------------------------------+----------

{'runId': 'AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-04T04:00:25.746495Z',
 'endTimeUtc': '2021-07-04T04:49:09.155826Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project-experiment","subscription_id":"2d391f5f-13ec-4301-9d0f-ac4301a018d8","resource_group":"myresourcegroup","workspace_name":"azudacityml01","region":"eastus2","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"AU

In [19]:
list(experiment.get_runs())[0].get_details()

{'runId': 'AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-04T04:00:25.746495Z',
 'endTimeUtc': '2021-07-04T04:49:09.155826Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project-experiment","subscription_id":"2d391f5f-13ec-4301-9d0f-ac4301a018d8","resource_group":"myresourcegroup","workspace_name":"azudacityml01","region":"eastus2","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"AU

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [63]:
best_run, model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.31.0, current version:1.30.0
Package:azureml-core, training version:1.31.0, current version:1.30.0
Package:azureml-dataprep, training version:2.18.0, current version:2.15.1
Package:azureml-dataprep-native, training version:36.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.16.0, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.31.0, current version:1.30.0
Package:azureml-defaults, training version:1.31.0, current version:1.30.0
Package:azureml-interpret, training version:1.31.0, current version:1.30.0
Package:azureml-mlflow, training version:1.31.0, current version:1.30.0
Package:azureml-pipeline-core, training version:1.31.0, current version:1.30.0
Package:azureml-telemetry, training version:1.31.0, current version:1.30.0
Package:azureml-train-automl-client, training version:1.31.0, current version:1.30.0
Package:azureml-train-automl-runtime, training version:1.31.0, current ver

In [56]:
!pip install --upgrade azureml-automl-core

     |████████████████████████████████| 208 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 39.4 MB 21 kB/s  eta 0:00:01
     |████████████████████████████████| 2.2 MB 47.0 MB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 51.3 MB/s eta 0:00:01
ERROR: azureml-widgets 1.30.0 has requirement azureml-core~=1.30.0, but you'll have azureml-core 1.31.0 which is incompatible.
ERROR: azureml-widgets 1.30.0 has requirement azureml-telemetry~=1.30.0, but you'll have azureml-telemetry 1.31.0 which is incompatible.
ERROR: azureml-train-core 1.30.0 has requirement azureml-core~=1.30.0, but you'll have azureml-core 1.31.0 which is incompatible.
ERROR: azureml-train-core 1.30.0 has requirement azureml-telemetry~=1.30.0, but you'll have azureml-telemetry 1.31.0 which is incompatible.
ERROR: azureml-train-automl 1.30.0 has requirement azureml-automl-core~=1.30.0, but you'll have azureml-automl-core 1.31.0

In [66]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":4,"CurrentNodeCount":4}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967_ModelExplain',
 'model_explanation': 'True'}

In [68]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'balanced_accuracy': 0.7229355564945195,
 'average_precision_score_macro': 0.7997492970053722,
 'matthews_correlation': 0.4472891398697703,
 'average_precision_score_micro': 0.8056763236208857,
 'f1_score_micro': 0.7229265868181665,
 'f1_score_macro': 0.7224803171250331,
 'AUC_weighted': 0.8010431017233097,
 'weighted_accuracy': 0.7601394920641226,
 'AUC_macro': 0.8010431017233097,
 'average_precision_score_weighted': 0.7997655518861818,
 'accuracy': 0.7229265868183272,
 'precision_score_micro': 0.7229265868181634,
 'recall_score_macro': 0.7229355564945195,
 'recall_score_micro': 0.7229265868181696,
 'norm_macro_recall': 0.44587111298961235,
 'f1_score_weighted': 0.7224871889288662,
 'log_loss': 0.536808070189603,
 'recall_score_weighted': 0.7229265868181696,
 'precision_score_weighted': 0.7243786362763155,
 'AUC_micro': 0.8033226480646501,
 'precision_score_macro': 0.7243558430543688,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_cbf39a66-adfa-434e-974e-c5fcdd11496

In [69]:
best_run.properties

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Stack_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Stack_Ensemble__","objects":[{"module":"azureml.train.automl.stack_ensemble","class_name":"StackEnsemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-project-experiment\',\'compute_target\':\'capstone-cluster\',\'subscription_id\':\'2d391f5f-13ec-4301-9d0f-ac4301a018d8\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967_29","experiment_name":"capstone-project-experiment","workspace_name":"azudacityml01","subscription_id":"2d391f5f-13ec-4301-9d0f-ac4301a018d8","resource_group_name":"myresourcegroup"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '29',
 '_aml_system_scenario_identification': 'Remote

In [70]:
automl_model_name = best_run.properties['model_name']
print('Best AutoML model name: ' + automl_model_name,
      'Best AutoML model run: ' + str(best_run),
      'Best AutoML model specification: ' + str(model), sep = '\n\n')

Best AutoML model name: AutoMLcbf39a66a29

Best AutoML model run: Run(Experiment: capstone-project-experiment,
Id: AutoML_cbf39a66-adfa-434e-974e-c5fcdd114967_29,
Type: azureml.scriptrun,
Status: Completed)

Best AutoML model specification: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=None, reg_alpha=1, reg_lambda=1, subsample=0.29736842105263156))], verbose=False))], meta_learner=LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False, fit_intercept=True, intercept_scaling=1.0, l1_ratios=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', random_state=None, refit=True, scoring=<azureml.automl.runtime.st

In [71]:
model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('stackensembleclassifier',
  StackEnsembleClassifier(
      base_learners=[('23', Pipeline(memory=None,
               steps=[('standardscalerwrapper',
                       StandardScalerWrapper(
          copy=True,
          with_mean=False,
          with_std=False
      )),
                      ('xgboostclassifier',
                       XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=8, max_leaves=7, n_estimators=100, n_jobs=-1, objective='reg:logistic', problem_info=ProblemInfo(
          dataset_samples=354663,
          dataset_features...
          iteration_timeout_mode=0,
          iteration_timeout_param=Non

In [72]:
#TODO: Save the best model

# Register and save the best model
automl_model_registered = remote_run.register_model(model_name='default_automl_model',
                        tags={'Training context':'Auto ML'}
                        )

automl_model_registered.download(target_dir="outputs_automl", exist_ok=True)

'outputs_automl/model.pkl'

In [20]:
from azureml.core import Model

# List registered models
for model in Model.list(ws):
    print('Name: ', model.name, 'version: ', model.version)

    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    
    print('\n')

Name:  default_automl_model version:  7
	 Training context : Auto ML


Name:  default_automl_model version:  6
	 Training context : Auto ML


Name:  default_automl_model version:  5
	 Training context : Auto ML


Name:  default_automl_model version:  4
	 Training context : Auto ML


Name:  default_automl_model version:  3
	 Training context : Auto ML
	 AUC : 0.8010431017233097
	 Accuracy : 0.7229265868183272


Name:  default_automl_model version:  2


Name:  default_automl_model version:  1




In [74]:
!ls -l outputs_automl

total 2819
-rwxrwxrwx 1 root root 2886347 Jul  4 21:55 model.pkl


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

In [22]:
myenv = Environment(name="capstoneenv")
myenv.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210531.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "capstoneenv",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [24]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=4, 
                                                enable_app_insights=True,
                                                description='Predict default score')

model = Model(ws, 'default_automl_model')

env_deploy = Environment.get(workspace=ws, name='capstoneenv')

inference_config = InferenceConfig(entry_script="score.py", environment=env_deploy)

service = Model.deploy(workspace=ws, 
                       name='default-automl-v1', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

In [ ]:
#Model.get_model_path('default_automl_model')

In [25]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-05 00:07:27+00:00 Creating Container Registry if not exists.
2021-07-05 00:07:27+00:00 Registering the environment.
2021-07-05 00:07:28+00:00 Use the existing image.
2021-07-05 00:07:28+00:00 Generating deployment configuration.
2021-07-05 00:07:29+00:00 Submitting deployment to compute.
2021-07-05 00:07:32+00:00 Checking the status of deployment default-automl-v1..
2021-07-05 00:08:55+00:00 Checking the status of inference endpoint default-automl-v1.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 6e0f893f-a115-4ece-98bc-c571b2a538a0
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: default-automl-v1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image azudacitycr01.azurecr.io/azureml/azureml_2dc4016849bfdcec9f9d4bb2200f06f4 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more info

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 6e0f893f-a115-4ece-98bc-c571b2a538a0
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: default-automl-v1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image azudacitycr01.azurecr.io/azureml/azureml_2dc4016849bfdcec9f9d4bb2200f06f4 locally. Please refer to https://

In [80]:
print(service.get_logs())

None


In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service